### 2 Clean Data

In [46]:
import pandas as pd
import numpy as np
import statistics
import datetime


In [47]:
bt=pd.read_csv("bank_transactions.csv")      # Import the dataset
bt.dropna(inplace=True)                      # Dropping missing values 

In [48]:
# convert to timedate
bt.CustomerDOB = pd.to_datetime(bt.CustomerDOB)
bt.TransactionDate = pd.to_datetime(bt.TransactionDate)

In [49]:
def zeros(x):
    l = len(x)
    if l != 6:
        x = '0' * (6-l) + x
    return x

In [50]:
bt.TransactionTime = list(map(str,bt.TransactionTime))
bt.TransactionTime = list(map(zeros,bt.TransactionTime))
bt.TransactionTime = bt.TransactionTime.apply(lambda x: f'{x[:2]}:{x[2:4]}:{x[4:]}')
bt.TransactionTime = pd.to_datetime(bt.TransactionTime, format='%H:%M:%S')

In [51]:
bt.loc[bt.CustomerDOB.dt.year > 2000, 'CustomerDOB'] = bt.loc[bt.CustomerDOB.dt.year > 2000, 'CustomerDOB'] - pd.DateOffset(years = 100)
bt.drop(bt[bt.CustomerDOB.dt.year == 1800].index, axis=0, inplace=True)

In [52]:
#calculating customer age
bt['CustomerAge'] = (( pd.to_datetime('today') - bt.CustomerDOB ) / np.timedelta64(1, 'Y')).round(0)
bt['CustomerAge'] = bt['CustomerAge'].astype(int)

In [53]:
bt['TransactionAmount (INR)'] = bt['TransactionAmount (INR)'].astype(int)
bt['Utilisation'] = bt['CustAccountBalance'] - bt['TransactionAmount (INR)']

In [54]:
#create new key:
bt["NewID"] = bt["CustomerID"] + "-" + bt["CustomerDOB"].astype(str)

In [55]:
#to save
bt.to_pickle("bank.pkl")

In [56]:
#to load
bt = pd.read_pickle("bank.pkl")

### 2.1 Getting your data + feature engineering

In [57]:
#Define an empty dataframe
Features = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

#Number of transaction
Features["Num_of_Transaction"] = bt[["NewID","TransactionID"]].groupby("NewID").count()

#Transaction with balance > 100:
Features["Balance>100_USD"] = bt[["NewID","CustAccountBalance"]].groupby("NewID")["CustAccountBalance"].apply(lambda x: (x>100).sum())


#Average Amount of the transactions:
Features["Average_Amount"] = bt[["NewID","TransactionAmount (INR)"]].groupby("NewID").agg(Average_Amount=("TransactionAmount (INR)",np.mean))


#Average Balance:
Features["Average_Balance"] = bt[["NewID","CustAccountBalance"]].groupby("NewID").agg(Average_Balance=("CustAccountBalance",np.mean))


#Utilisation = Average difference between the balance and the transaction amount:
pd.options.mode.chained_assignment = None      #Avoid the red banner
bt["Utilisation"] = bt["CustAccountBalance"]-bt["TransactionAmount (INR)"]  #create "Utilisation" column
Features["Utilisation"] = bt[["NewID","Utilisation"]].groupby("NewID").agg(Utilisation=("Utilisation",np.mean))  #calculate the mean  

#Gender of the customer:  
Features["gender"]=bt[["NewID","CustGender"]].groupby("NewID").last()

#Most frequent location of the customer
Features["Most_Frequent_Location"] = bt[["NewID","CustLocation"]].groupby("NewID").agg(CustLocation =("CustLocation",statistics.mode))


In [61]:
Features

,Num_of_Transaction,Balance>100_USD,Average_Amount,Average_Balance,Utilisation,gender,Most_Frequent_Location
NewID,,,,,,,
C1010011-1983-05-08,1,1,356.0,120180.54,119824.54,M,NEW DELHI
C1010011-1992-08-19,1,1,4750.0,32500.73,27750.73,F,NOIDA
C1010012-1994-07-28,1,1,1499.0,24204.49,22705.49,M,MUMBAI
C1010014-1984-08-19,1,1,250.0,161848.76,161598.76,M,MUMBAI
C1010014-1992-04-06,1,1,1205.0,38377.14,37172.14,F,MUMBAI
...,...,...,...,...,...,...,...
C9099836-1990-12-24,1,1,691.0,133067.23,132376.23,M,BHIWANDI
C9099877-1996-09-06,1,1,222.0,96063.46,95841.46,M,BANGALORE
C9099919-1993-10-21,1,1,126.0,5559.75,5433.75,M,GUNTUR


In [62]:
#save
Features.to_pickle("bankf.pkl")

In [63]:
#load
Features = pd.read_pickle("bankf.pkl")

### Adding 20 additional features

* 1 Customer age
* 2 Maximum transaction of the customer
* 3 Minimun transaction of the customer
* 4 Important customer (Define as Average_Balance greater than 90% of the average balance of all customers
* 5 Teen ager (Define  20 years before the last transaction registered )
* 6 Class Age ( young <= 25 ; 26 < grown-up <= 60  ; elderly > 60 )
* 7 Transaction with balance < 50:
* 8 Date of birth of the costumer



In [64]:
Features["Costumer_age"] = bt[["NewID","CustomerAge"]].groupby("NewID").last()

In [65]:
Features["Max_transaction"] = bt[["NewID","TransactionAmount (INR)"]].groupby("NewID").max()

In [66]:
Features["Min_transaction"] = bt[["NewID","TransactionAmount (INR)"]].groupby("NewID").min()

In [67]:
Features["Important_costumer"] = Features["Average_Balance"] > Features["Average_Balance"].quantile(.9)

In [68]:
# Qua si è settata la data dopo la quale le persone nate dopo sono teen ager
d1 = datetime.datetime(max(bt.TransactionDate).year - 20 ,max(bt.TransactionDate).month , max(bt.TransactionDate).day)
bt['teen'] = bt.CustomerDOB > d1
# Creazione della variabile
Features['teen_ager'] = bt[["NewID","teen"]].groupby("NewID").last()



In [69]:
# Copia di una varaibile che sarà modificata
Features["Class_Age"]  = Features["Costumer_age"]

# Set una singola età per ogni classe

Features.Class_Age[ (Features["Class_Age"] > 25) &  (Features["Class_Age"] <= 60) ]   = 50
Features.Class_Age[ Features["Class_Age"] <= 25]  = 20
Features.Class_Age[ Features["Class_Age"] > 60 ]  = 70

#Creazione della classe

Features.Class_Age[ Features["Class_Age"] == 20 ]  = "young"
Features.Class_Age[ Features["Class_Age"] == 50 ]  = "adult"
Features.Class_Age[ Features["Class_Age"] == 50 ]  = "elderly"




In [70]:
#Transaction with balance < 50:

Features["Balance<50_USD"] = bt[["NewID","CustAccountBalance"]].groupby("NewID")["CustAccountBalance"].apply(lambda x: (x<50).sum())


In [71]:
Features["Date_of_birth"] = bt[["NewID","CustomerDOB"]].groupby("NewID").last()

In [129]:
Features

,Num_of_Transaction,Balance>100_USD,Average_Amount,Average_Balance,Utilisation,gender,Most_Frequent_Location,Costumer_age,Max_transaction,Min_transaction,Important_costumer,Class_Age,teen_ager,Balance<50_USD,Date_of_birth
CustomerID,,,,,,,,,,,,,,,
C1010011,2,2,2553.0,76340.635,73787.635,M,NOIDA,40,4750,356,False,adult,False,0,1983-05-08
C1010012,1,1,1499.0,24204.490,22705.490,M,MUMBAI,28,1499,1499,False,adult,False,0,1994-07-28
C1010014,2,2,727.5,100112.950,99385.450,M,MUMBAI,38,1205,250,False,adult,False,0,1984-08-19
C1010018,1,1,30.0,496.180,466.180,F,CHAMPARAN,32,30,30,False,adult,False,0,1990-05-29
C1010024,1,1,5000.0,87058.650,82058.650,M,KOLKATA,57,5000,5000,False,adult,False,0,1965-06-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C9099836,1,1,691.0,133067.230,132376.230,M,BHIWANDI,32,691,691,False,adult,False,0,1990-12-24
C9099877,1,1,222.0,96063.460,95841.460,M,BANGALORE,26,222,222,False,adult,False,0,1996-09-06
C9099919,1,1,126.0,5559.750,5433.750,M,GUNTUR,29,126,126,False,adult,False,0,1993-10-21
